# Simulator

In [1]:
using PyCall
ipd = pyimport("IPython.display")

function getwav(filename::AbstractString)::AbstractArray
    raw = UInt8[]
    open(filename) do f
        readbytes!(f, raw, Inf)
    end
    header = raw[1:44]
    data = reinterpret(Int16, raw[45:end])
    return data
end

getwav (generic function with 1 method)

## Julia DLL interface

In [66]:
using Libdl

const MIDI_CC_PRESET = 0x50
const MIDI_CC_VOLUME = 0x46
const MIDI_CC_CUTOFF_VELOCITY = 0x0a
const MIDI_CC_SUSTAIN = 0x40
const MIDI_CC_CUTOFF = 0x4a
const MIDI_CC_RESONANCE = 0x47
const MIDI_CC_AMP_ATTACK = 0x53
const MIDI_CC_AMP_DECAY = 0x59
const MIDI_CC_AMP_SUSTAIN = 0x55
const MIDI_CC_AMP_RELEASE = 0x52

_so = nothing

function init()
    global _so
    _so = dlopen("../simulator/simulator.so")
    resp = ccall(dlsym(_so, :sim_init), Int32, ())
    if resp != 0
        error("init failed")
    end
end

function close()
    global _so
    resp = ccall(dlsym(_so, :sim_close), Int32, ())
    if resp != 0
        error("close failed")
    end
    dlclose(_so)
    _so = nothing
end

function tick()
    resp = ccall(dlsym(_so, :sim_tick), Int32, ())
    if resp != 0
        error("tick failed")
    end
end

function enqueue(status, data0, data1)
    resp = ccall(
        dlsym(_so, :_midi_enqueue),
        Int32,
        (UInt8, UInt8, UInt8),
        status, data0, data1
    )
    if resp != 0
        error("enqueue failed")
    end
end

control(param, value) = enqueue(0xb0, param, value)
noteon(note, vel) = enqueue(0x90, note, vel)
noteoff(note) = enqueue(0x80, note, 0)


noteoff (generic function with 2 methods)

## Simulation

In [73]:
init()

"""
# Control messages
control(1000, MIDI_CC_PRESET, 0)
control(1010, MIDI_CC_CUTOFF_VELOCITY, 0)
control(1020, MIDI_CC_CUTOFF, 60)
control(1030, MIDI_CC_AMP_ATTACK, 1)

# Notes
noteon(10000, 24, 127)
noteon(20010, 31, 127)
noteon(30020, 36, 127)

noteoff(80000, 24)
noteoff(80010, 31)
noteoff(80020, 36)
"""

for n=1:200000
    if n==1000 control(MIDI_CC_PRESET, 0) end
    if n==1010 control(MIDI_CC_CUTOFF_VELOCITY, 0) end
    if n==1020 control(MIDI_CC_CUTOFF, 50) end
    
    if n==10000 noteon(24, 127) end
    if n==20000 noteon(31, 127) end
    if n==30000 noteon(36, 127) end
    
    if n==80000 noteoff(24) end
    if n==80010 noteoff(31) end
    if n==80020 noteoff(36) end
    tick()
end

close()

ipd.Audio("output.wav")

b0 50 00
b0 0a 00
b0 4a 32
90 18 7f
90 1f 7f
90 24 7f
80 18 00
80 1f 00
80 24 00


PyObject <IPython.lib.display.Audio object>